In [6]:
import os
import cv2
import glob
import numpy as np

In [7]:
def find_blue_boxes(image, color):
    # 이미지를 HSV 색 공간으로 변환
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # 색 경계를 찾기 위한 마스크 생성
    lower_color = color[0]
    upper_color = color[1]
    mask = cv2.inRange(hsv, lower_color, upper_color)

    # 마스크를 이용하여 색 영역 추출
    blue_area = cv2.bitwise_and(image, image, mask=mask)

    # 색 영역의 경계 좌표를 찾음
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    ct_list = []
    for ct in contours:
        ct_count = np.array(ct).shape[0]
        if ct_count > 50 and ct_count < 200:
            ct_list.extend(ct)

    ct_list = np.array(ct_list)

    if len(ct_list) == 0:
        return None, None
    else:
        # 색 영역 경계를 기반으로 이미지를 자름
        # x, y, w, h = cv2.boundingRect(contours[max_ct])
        x, y, w, h = cv2.boundingRect(ct_list)
        cropped_image = image[y:y+h, x:x+w]

        return blue_area, cropped_image

In [8]:
def fill_red_boxes(image, color, color2):
    # 이미지를 HSV 색 공간으로 변환
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    lower_color = color[0]
    upper_color = color[1]
    lower_color2 = color2[0]
    upper_color2 = color2[1]

    mask1 = cv2.inRange(hsv, lower_color, upper_color)
    mask2 = cv2.inRange(hsv, lower_color2, upper_color2)
    mask = cv2.bitwise_or(mask1, mask2)

    # 마스크를 이용하여 색 영역 추출
    red_area = cv2.bitwise_and(image, image, mask=mask)

    # 색 영역의 경계 좌표를 찾음
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        hull = cv2.convexHull(contour)
        cv2.drawContours(image, [hull], 0, (0, 0, 255), -1)

    image[np.where((image != [0, 0, 255]).all(axis=2))] = [0, 0, 0]

    return red_area, image

In [9]:
def get_target(image_path, blue, red):
    image = cv2.imread(image_path)
    _, folder_name, file_name = image_path.split("/")

    _, cropped_image = find_blue_boxes(image, blue)
    if cropped_image is None:
        print(image_path)
        return None
    else:
        _, fill_image = fill_red_boxes(cropped_image, red[0], red[1])
        if np.count_nonzero(fill_image) > 1000:
            if not os.path.exists('test/' + folder_name):
                os.makedirs('test/' + folder_name)

            # output_path = 'test/' + folder_name + '/crop_' + file_name
            # cv2.imwrite(output_path, cropped_image)

            output_path = 'test/' + folder_name + '/' + file_name
            cv2.imwrite(output_path, fill_image)
        else:
            print(image_path)

In [10]:
# 상위 폴더 경로
folder_path = "data"
images = []

# Blue
lower_blue = np.array([100, 50, 50])
upper_blue = np.array([130, 255, 255])
blue_color = [lower_blue, upper_blue]

# Red
lower_red1 = np.array([0, 20, 20])
upper_red1 = np.array([20, 255, 255])
lower_red2 = np.array([130, 20, 20])
upper_red2 = np.array([255, 255, 255])
red1 = [lower_red1, upper_red1]
red2 = [lower_red2, upper_red2]
red_color = [red1, red2]

# 모든 하위 폴더 순회
for root, dirs, files in os.walk(folder_path):
    for dir in dirs:
        # 하위 폴더 내의 이미지 파일 목록 가져오기
        subfolder_path = os.path.join(root, dir)
        image_files = glob.glob(subfolder_path + "/*.jpg")  # .jpg 확장자를 가진 이미지 파일들을 가져옴

        # 이미지 파일들을 읽어오기
        for image_file in image_files:
            get_target(image_file, blue_color, red_color)

data/IF~13_01_24/F001-0012.jpg
data/IF~13_01_24/F001-0001.jpg
data/IF~13_01_24/F001-0011.jpg
data/IF~13_01_24/F001-0006.jpg
data/IF~13_54_05/F001-0001.jpg
data/IF~13_54_05/F001-0006.jpg
data/DN8~13_08_57/F002-0028.jpg
data/DN8~13_08_57/F001-0004.jpg
data/DN8~13_08_57/F002-0033.jpg
data/DN8~13_08_57/F003-0007.jpg
data/DN8~13_08_57/F002-0007.jpg
data/DN8~13_08_57/F002-0032.jpg
data/DN8~13_08_57/F002-0013.jpg
data/DN8~13_08_57/F002-0017.jpg
data/DN8~13_08_57/F002-0003.jpg
data/DN8~13_08_57/F002-0026.jpg
data/DN8~13_08_57/F002-0020.jpg
data/DN8~13_08_57/F002-0018.jpg
data/DN8~13_08_57/F002-0027.jpg
data/DN8~13_08_57/F002-0002.jpg
data/DN8~13_08_57/F001-0008_NG_CAM300.jpg
data/DN8~13_08_57/F002-0034.jpg
data/DN8~13_08_57/F004-0006.jpg
data/DN8~13_08_57/F002-0025.jpg
data/DN8~13_08_57/F001-0001.jpg
data/DN8~13_08_57/F002-0012.jpg
data/DN8~13_08_57/F002-0019.jpg
data/DN8~13_08_57/F002-0029.jpg
data/DN8~13_08_57/F004-0001.jpg
data/DN8~13_08_57/F002-0024.jpg
data/DN8~13_08_57/F001-0020.jpg
data